In [1]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
messages = [
    {"role": "user", "content": "What is the approximate number of luffas that a fully grown luffa tree can produce in a single growing season?"},
    {"role": "assistant", "content": "A mature luffa tree can typically produce several clusters of fruit throughout the growing season. Each cluster can contain several luffas, and a fully grown luffa tree may produce between 10 to 20 clusters of fruit in a single growing season."},
    {"role": "user", "content": "Let's go plant some luffa trees together"}
]
model_name = '/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
    )
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds
generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1935: UserWarning: You are calling .generate() with the 

In [4]:
print(decoded[0])

<s> [INST] What is the approximate number of luffas that a fully grown luffa tree can produce in a single growing season? [/INST]A mature luffa tree can typically produce several clusters of fruit throughout the growing season. Each cluster can contain several luffas, and a fully grown luffa tree may produce between 10 to 20 clusters of fruit in a single growing season.</s>  [INST] Let's go plant some luffa trees together [/INST] Sure! Luffa trees are typically easy to grow and thrive in a variety of soil types and climates. To plant luffa trees, you'll need to follow these general steps:

1. Choose a location: Select an area where the tree will receive ample sunlight and have enough space to grow.
2. Prepare the soil: Loosen the soil with a garden fork to about 12 inches deep, as luffa trees require well-drained soil.
3. Plant the seeds: Luffa seeds are typically planted about 1 inch deep and 1-2 feet apart in rows that are about 2-3 feet apart.
4. Care for the seedlings: Water the se

In [6]:
!pip install -U langchain-community

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.3 MB/s eta 0:00:00a 0:00:01
  Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.3.0
    Uninstalling pydantic-2.3.0:
      Successfully uninstalled pydantic-2.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi 0.98.0 requires pydantic!=1.8,!=1.8.1,<2.0.0,>=1.7.4, but you have pydantic 2.9.2 which is incompatible.
ydata-profiling 4.3.1 requires pydantic<2,>=1.8.1, but you have pydantic 2.9.2 which is incompatible.
ydata-profiling 4.3.1 requires scipy<1.11,>=1.4.1, but you have scipy 1.11.2 which is incompatible.


Credit:
 - Adapted from https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1

In [8]:

# Import necessary libraries
import time  # Import time module for tracking time
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

# Start tracking the time for loading PDFs
start_time = time.time()

# Load PDF documents from the specified directory
loader = PyPDFDirectoryLoader("/kaggle/input/ncert-dataset")  # Change this path to your uploaded PDF folder
data = loader.load()

# Record the time taken to load PDFs
pdf_load_time = time.time() - start_time
print(f"Time taken to load PDFs: {pdf_load_time:.2f} seconds")

# Step 05: Split the Extracted Data into Text Chunks
start_time = time.time()  # Start time tracking for text splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

# Record the time taken to split text
split_time = time.time() - start_time
print(f"Time taken to split text: {split_time:.2f} seconds")

# Step 06: Download the Embeddings
start_time = time.time()  # Start time tracking for embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

# Record the time taken for embedding generation
embedding_time = time.time() - start_time
print(f"Time taken to generate embeddings: {embedding_time:.2f} seconds")



Time taken to load PDFs: 5.77 seconds
Time taken to split text: 0.00 seconds
Time taken to generate embeddings: 1.00 seconds


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Change this to the model path or name from Hugging Face
model_name = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"  # Use a valid path or model name from Hugging Face

# Load tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# Tokenize input
messages = [{"role": "user", "content": "What is linear sound?"}]
input_ids = tokenizer.encode(messages[0]['content'], return_tensors="pt").to("cuda")

# Generate text response
with torch.no_grad():
    outputs = model.generate(input_ids, max_new_tokens=50)

# Decode output
decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_output)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What is linear sound?
A: Linear sound is a type of sound that is produced by vibrations that move in a straight line. This type of sound is often described as being clear and crisp, and can be heard in a variety of environments, including music


In [17]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import FAISS

# Step 1: Load Pre-trained LLM (same as before)
model_name = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"  # Change to your model path or Hugging Face model name
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [18]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

hf_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens = 2000)
# gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_new_tokens=200)

llm = HuggingFacePipeline(pipeline=hf_pipeline)

# Step 4: Create a retriever from the FAISS vector store
retriever = vector_store.as_retriever(search_kwargs={"k": 2})  # Retrieve top 2 relevant documents

# Step 5: Set up the Retrieval-Augmented Generation (RAG) chain using the wrapped LLM
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",  # Use "stuff" or other types depending on your needs
    return_source_documents=True  # Optionally return retrieved documents
)

# Step 6: Query the RAG system
query = "What is linear sound?"  # Example query from user

# You may want to set `max_length` or `max_new_tokens` in your pipeline call
result = qa_chain({"query": query, "max_new_tokens": 10000})  # Set max_new_tokens as needed

# Print the generated response
print("Generated Response:", result['result'])

# Optionally, print the retrieved documents (for debug purposes)
for i, doc in enumerate(result['source_documents']):
    print(f"\nRetrieved Document {i + 1}:\n", doc.page_content)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Generated Response: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

SOUND 131How the brain interprets the frequency
of an emitted sound is called its pitch. The
faster the vibration of the source, the
higher is the frequency and the higher is
the pitch, as shown in Fig. 11.7. Thus, a
high pitch sound corresponds to more
number of compressions and rarefactions
passing a fixed point per unit time.
Objects of different sizes and conditions
vibrate at different frequencies to produce
sounds of different pitch.
The magnitude of the maximum
disturbance in the medium on either side of
the mean value is called the amplitude of the
wave. It is usually represented by the letter A,as shown in Fig. 11.6(c). For sound its unit
will be that of density or pressure.
The loudness or softness of a sound is
determined basically by its amplitude. The
amplitude of the sound wave depends upon


In [ ]:
# Create RetrievalQA instance
start_time = time.time()  # Start time tracking for QA chain creation
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

# Record the time taken to create the QA chain
qa_creation_time = time.time() - start_time
print(f"Time taken to create QA chain: {qa_creation_time:.2f} seconds")

# Define your query
query = "What is linear sound"

# Run the query and get the answer
start_time = time.time()  # Start time tracking for running the query
response = qa.run(query)

# Record the time taken to run the query
query_time = time.time() - start_time
print(f"Time taken to run the query: {query_time:.2f} seconds")

# Output the response
print("Query Response:")
print(response)